In [1]:
import os
import glob
import json
import random
import numpy as np
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from keras_preprocessing.sequence import pad_sequences

2023-11-05 10:57:23.693597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
files = glob.glob("/home/imcurie/ml/chinese-poetry/全唐诗/poet.tang.*.json")
write_dir = "/home/imcurie/ml/neuron-network-learning/data/poetry"
write_file_name = "poetry.txt"
full_path = os.path.join(write_dir, write_file_name)

if not os.path.exists(full_path):
    for file in files:
        with open(file, "r", encoding="utf-8") as f_read:
            json_datas = json.load(f_read)
            for json_data in json_datas:
                with open(full_path, "a", encoding="utf-8") as f_write:
                    poetry = ''
                    for paragraph in json_data["paragraphs"]:
                        poetry += paragraph
                    f_write.write(poetry + "\n")

datas = []
with open(full_path, 'r', encoding="utf-8") as f:
     while line := f.readline():
        datas.append(line.strip('\n'))
        
for i in range(4):
    print(datas[i])

一身遶千山，遠作行路人。未遂東吳歸，暫出西京塵。仲宣荆州客，今余竟陵賓。往蹟雖不同，託意皆有因。商嶺莓苔滑，石坂上下頻。江漢沙泥潔，永日光景新。獨淚起殘夜，孤吟望初晨。驅馳竟何事，章句依深仁。
楚山爭蔽虧，日月無全輝。楚路饒迴惑，旅人有迷歸。騏驥思北首，鷓鴣願南飛。我懷京洛遊，未厭風塵衣。
衆䖟聚病馬，流血不得行。後路起夜色，前山聞虎聲。此時遊子心，百尺風中旌。
分拙多感激，久遊遵長途。經過湘水源，懷古方踟躕。舊稱楚靈均，此處殞忠軀。側聆故老言，遂得旌賢愚。名參君子場，行爲小人儒。騷文衒貞亮，體物情崎嶇。三黜有慍色，即非賢哲模。五十爵高秩，謬膺從大夫。胸襟積憂愁，容鬢復彫枯。死爲不弔鬼，生作猜謗徒。吟澤潔其身，忠節寧見輸。懷沙滅其性，孝行焉能俱。且聞善稱君，一何善自殊。且聞過稱己，一何過不渝。悠哉風土人，角黍投川隅。相傳歷千祀，哀悼延八區。如今聖明朝，養育無羈孤。君臣逸雍熙，德化盈紛敷。巾車徇前侶，白日猶昆吾。寄君臣子心，戒此真良圖。


In [3]:
poetry_num = len(datas)
for i in range(poetry_num):
    datas[i] = ' '.join(datas[i])

for i in range(poetry_num):
    datas[i] = '<SOS> ' + datas[i] + ' <EOS>'
    
for i in range(4):
    print(datas[i])

<SOS> 一 身 遶 千 山 ， 遠 作 行 路 人 。 未 遂 東 吳 歸 ， 暫 出 西 京 塵 。 仲 宣 荆 州 客 ， 今 余 竟 陵 賓 。 往 蹟 雖 不 同 ， 託 意 皆 有 因 。 商 嶺 莓 苔 滑 ， 石 坂 上 下 頻 。 江 漢 沙 泥 潔 ， 永 日 光 景 新 。 獨 淚 起 殘 夜 ， 孤 吟 望 初 晨 。 驅 馳 竟 何 事 ， 章 句 依 深 仁 。 <EOS>
<SOS> 楚 山 爭 蔽 虧 ， 日 月 無 全 輝 。 楚 路 饒 迴 惑 ， 旅 人 有 迷 歸 。 騏 驥 思 北 首 ， 鷓 鴣 願 南 飛 。 我 懷 京 洛 遊 ， 未 厭 風 塵 衣 。 <EOS>
<SOS> 衆 䖟 聚 病 馬 ， 流 血 不 得 行 。 後 路 起 夜 色 ， 前 山 聞 虎 聲 。 此 時 遊 子 心 ， 百 尺 風 中 旌 。 <EOS>
<SOS> 分 拙 多 感 激 ， 久 遊 遵 長 途 。 經 過 湘 水 源 ， 懷 古 方 踟 躕 。 舊 稱 楚 靈 均 ， 此 處 殞 忠 軀 。 側 聆 故 老 言 ， 遂 得 旌 賢 愚 。 名 參 君 子 場 ， 行 爲 小 人 儒 。 騷 文 衒 貞 亮 ， 體 物 情 崎 嶇 。 三 黜 有 慍 色 ， 即 非 賢 哲 模 。 五 十 爵 高 秩 ， 謬 膺 從 大 夫 。 胸 襟 積 憂 愁 ， 容 鬢 復 彫 枯 。 死 爲 不 弔 鬼 ， 生 作 猜 謗 徒 。 吟 澤 潔 其 身 ， 忠 節 寧 見 輸 。 懷 沙 滅 其 性 ， 孝 行 焉 能 俱 。 且 聞 善 稱 君 ， 一 何 善 自 殊 。 且 聞 過 稱 己 ， 一 何 過 不 渝 。 悠 哉 風 土 人 ， 角 黍 投 川 隅 。 相 傳 歷 千 祀 ， 哀 悼 延 八 區 。 如 今 聖 明 朝 ， 養 育 無 羈 孤 。 君 臣 逸 雍 熙 ， 德 化 盈 紛 敷 。 巾 車 徇 前 侶 ， 白 日 猶 昆 吾 。 寄 君 臣 子 心 ， 戒 此 真 良 圖 。 <EOS>


In [4]:
tokenizer = Tokenizer(num_words=10000, char_level=False, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(datas)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

## 使用word_index属性查看每个词对应的编码
## 使用word_counts属性查看每个词对应的频数
for ii, iterm in enumerate(tokenizer.word_index.items()):
    if ii < 10:
        print(iterm)
    else:
        break
print("===================")  
for ii, iterm in enumerate(tokenizer.word_counts.items()):
    if ii < 10:
        print(iterm)
    else:
        break

('，', 1)
('。', 2)
('<sos>', 3)
('<eos>', 4)
('不', 5)
('人', 6)
('一', 7)
('無', 8)
('山', 9)
('風', 10)
('<sos>', 57607)
('一', 20277)
('身', 5325)
('遶', 748)
('千', 6922)
('山', 18212)
('，', 268814)
('遠', 6429)
('作', 8982)
('行', 9130)


In [5]:
# 按照空格进行分词
datas_split = []
for i in range(poetry_num):
    datas_split.append(datas[i].split(' '))

In [6]:
x_seq_ls = []
y_seq_ls = []
decoder_output_ls = []

for i in tqdm(range(poetry_num)):
    
    if len(datas_split[i]) - 21 <= 0:
        continue
    
    start = random.randint(0, len(datas_split[i]) - 11)
    end = start + 10

    x_seq_ls.append(datas_split[i][start:end])
    y_seq_ls.append(datas_split[i][end:end+10])
    decoder_output_ls.append(datas_split[i][end+1:end+11])

for i in range(4):
    print(x_seq_ls[i],'\n',y_seq_ls[i],'\n',decoder_output_ls[i],'\n\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57607/57607 [00:00<00:00, 167340.84it/s]

['潔', '，', '永', '日', '光', '景', '新', '。', '獨', '淚'] 
 ['起', '殘', '夜', '，', '孤', '吟', '望', '初', '晨', '。'] 
 ['殘', '夜', '，', '孤', '吟', '望', '初', '晨', '。', '驅'] 


['輝', '。', '楚', '路', '饒', '迴', '惑', '，', '旅', '人'] 
 ['有', '迷', '歸', '。', '騏', '驥', '思', '北', '首', '，'] 
 ['迷', '歸', '。', '騏', '驥', '思', '北', '首', '，', '鷓'] 


['得', '行', '。', '後', '路', '起', '夜', '色', '，', '前'] 
 ['山', '聞', '虎', '聲', '。', '此', '時', '遊', '子', '心'] 
 ['聞', '虎', '聲', '。', '此', '時', '遊', '子', '心', '，'] 


['過', '湘', '水', '源', '，', '懷', '古', '方', '踟', '躕'] 
 ['。', '舊', '稱', '楚', '靈', '均', '，', '此', '處', '殞'] 
 ['舊', '稱', '楚', '靈', '均', '，', '此', '處', '殞', '忠'] 




In [7]:
x_token = tokenizer.texts_to_sequences(x_seq_ls)
y_token = tokenizer.texts_to_sequences(y_seq_ls)
decoder_output_token = tokenizer.texts_to_sequences(decoder_output_ls)

for i in range(4):
    print(x_token[i],'\n',y_token[i],'\n',decoder_output_token[i],'\n\n')

[1422, 1, 565, 11, 118, 299, 98, 2, 125, 341] 
 [185, 280, 54, 1, 214, 207, 135, 187, 712, 2] 
 [280, 54, 1, 214, 207, 135, 187, 712, 2, 845] 


[888, 2, 309, 84, 1276, 317, 2393, 1, 849, 6] 
 [12, 591, 39, 2, 3283, 2232, 101, 188, 279, 1] 
 [591, 39, 2, 3283, 2232, 101, 188, 279, 1, 2404] 


[47, 40, 2, 158, 84, 185, 54, 107, 1, 63] 
 [9, 124, 608, 71, 2, 31, 19, 167, 61, 29] 
 [124, 608, 71, 2, 31, 19, 167, 61, 29, 1] 


[178, 573, 21, 824, 1, 342, 132, 142, 2551, 2526] 
 [2, 169, 645, 309, 308, 1190, 1, 31, 62, 4551] 
 [169, 645, 309, 308, 1190, 1, 31, 62, 4551, 1293] 




In [8]:
# 导入补零需要的padding_seq
x_mat = pad_sequences(x_token, maxlen=10, padding='post', truncating='post')
y_mat = pad_sequences(y_token, maxlen=10, padding='post', truncating='post')
decoder_output_mat = pad_sequences(decoder_output_token, maxlen=10, padding='post', truncating='post')
decoder_output_mat = decoder_output_mat.reshape(decoder_output_mat.shape[0], decoder_output_mat.shape[1], 1)

for i in range(4):
    print(x_mat[i],'\n',y_mat[i],'\n\n')

[1422    1  565   11  118  299   98    2  125  341] 
 [185 280  54   1 214 207 135 187 712   2] 


[ 888    2  309   84 1276  317 2393    1  849    6] 
 [  12  591   39    2 3283 2232  101  188  279    1] 


[ 47  40   2 158  84 185  54 107   1  63] 
 [  9 124 608  71   2  31  19 167  61  29] 


[ 178  573   21  824    1  342  132  142 2551 2526] 
 [   2  169  645  309  308 1190    1   31   62 4551] 




In [9]:
x_train, x_test, y_train, y_test, decoder_output_train, decoder_output_test = train_test_split(x_mat, y_mat, decoder_output_mat, test_size=0.3, random_state=0)

for i in range(4):
    print(x_train[i],'\n',y_train[i],'\n\n')

[ 894    1  454   22  339 2022  155    2   31   16] 
 [  25   36  561    1  985  756  171   57 1037    2] 


[2587  541  565    1  196  398   16  369    2 2325] 
 [ 845 1399 4292    1   14  916 2159 2007    2   41] 


[ 285  135   62    1 1434  734  501 4032  327  126] 
 [86  2  4  0  0  0  0  0  0  0] 


[   7 3209    1   83   29  146 1023   83    8  146] 
 [   2   77 1149  100   23 1818  741  671    1  869] 




In [10]:
x_train_tensor = torch.tensor(x_train)
x_test_tensor = torch.tensor(x_test)
y_train_tensor = torch.tensor(y_train)
y_test_tensor = torch.tensor(y_test)
decoder_output_train_tensor = torch.tensor(decoder_output_train).long()
decoder_output_test_tensor = torch.tensor(decoder_output_test).long()


for i in range(4):
    print(x_train_tensor[i],'\n',y_train_tensor[i],'\n\n')

tensor([ 894,    1,  454,   22,  339, 2022,  155,    2,   31,   16],
       dtype=torch.int32) 
 tensor([  25,   36,  561,    1,  985,  756,  171,   57, 1037,    2],
       dtype=torch.int32) 


tensor([2587,  541,  565,    1,  196,  398,   16,  369,    2, 2325],
       dtype=torch.int32) 
 tensor([ 845, 1399, 4292,    1,   14,  916, 2159, 2007,    2,   41],
       dtype=torch.int32) 


tensor([ 285,  135,   62,    1, 1434,  734,  501, 4032,  327,  126],
       dtype=torch.int32) 
 tensor([86,  2,  4,  0,  0,  0,  0,  0,  0,  0], dtype=torch.int32) 


tensor([   7, 3209,    1,   83,   29,  146, 1023,   83,    8,  146],
       dtype=torch.int32) 
 tensor([   2,   77, 1149,  100,   23, 1818,  741,  671,    1,  869],
       dtype=torch.int32) 




In [11]:
SOS_token = tokenizer.word_index['<sos>']
EOS_token = tokenizer.word_index['<eos>']
PAD_token = tokenizer.word_index['<pad>']
SOS_token,EOS_token,PAD_token

(3, 4, 0)

In [12]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 初始化Shape为(max_len, d_model)的PE (positional encoding)
        pe = torch.zeros(max_len, d_model)
        # 初始化一个tensor [[0, 1, 2, 3, ...]]
        position = torch.arange(0, max_len).unsqueeze(1)
        # 这里就是sin和cos括号中的内容，通过e和ln进行了变换
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        # 计算PE(pos, 2i)
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算PE(pos, 2i+1)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 为了方便计算，在最外面在unsqueeze出一个batch
        pe = pe.unsqueeze(0)
        # 如果一个参数不参与梯度下降，但又希望保存model的时候将其保存下来
        # 这个时候就可以用register_buffer
        self.register_buffer("pe", pe)

    def forward(self, x):
        """
        x 为embedding后的inputs, 例如(1, 7, 128), batch size为1, 7个单词, 单词维度为128
        """
        # 将x和positional encoding相加，同时将它们移到相同的设备上
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)



In [13]:
class PoetTaskModel(nn.Module):

    def __init__(self, d_model=128):
        super(PoetTaskModel, self).__init__()

        # 词典数为10000
        self.embedding = nn.Embedding(num_embeddings=10000, embedding_dim=128)
        self.transformer = nn.Transformer(d_model=128, nhead = 2 ,num_encoder_layers=3, num_decoder_layers=3, dim_feedforward=128, batch_first=True)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0)

        # 定义最后的线性层，这里并没有用Softmax，因为没必要。
        # 因为后面的CrossEntropyLoss中自带了
        self.predictor = nn.Linear(128, 10000)

    def forward(self, src, tgt):

        # 生成mask
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size()[-1])
        src_key_padding_mask = PoetTaskModel.get_key_padding_mask(src)
        tgt_key_padding_mask = PoetTaskModel.get_key_padding_mask(tgt)

        src = self.embedding(src)
        tgt = self.embedding(tgt)
        src = self.positional_encoding(src)
        tgt = self.positional_encoding(tgt)

        # 将准备好的数据送给transformer
        out = self.transformer(src, tgt,
                               tgt_mask=tgt_mask,
                               src_key_padding_mask=src_key_padding_mask,
                               tgt_key_padding_mask=tgt_key_padding_mask)

        """
        这里直接返回transformer的结果。因为训练和推理时的行为不一样，
        所以在该模型外再进行线性层的预测。
        """
        return out

    @staticmethod
    def get_key_padding_mask(tokens):
        """
        用于key_padding_mask
        """
        key_padding_mask = torch.zeros(tokens.size())
        key_padding_mask[tokens == PAD_token] = -torch.inf
        return key_padding_mask

In [14]:
model = PoetTaskModel()
model

PoetTaskModel(
  (embedding): Embedding(10000, 128)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=128, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerDecoderLayer(
      

In [15]:
# 测试样本能否正常输入网络
src = x_train_tensor[:1]
tgt = y_train_tensor[:1]
out = model(src, tgt)
out.shape # torch.Size([1, 10, 10000]) 代表 batch_size, seq_len, vocab_size

torch.Size([1, 10, 128])

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

/home/imcurie/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from tqdm import tqdm

batch_size = 4
epochs = 1

for epoch in range(epochs):
    total_loss = 0

    for i in tqdm(range(0, len(x_train_tensor), batch_size)):
        src = x_train_tensor[i:i+batch_size]
        tgt = y_train_tensor[i:i+batch_size]
        tgt_y = decoder_output_train_tensor[i:i+batch_size]

        # 清空梯度
        optimizer.zero_grad()
        # 进行transformer的计算
        out = model(src, tgt)
        # 将结果送给最后的线性层进行预测
        out = model.predictor(out)
        # 计算loss
        loss = criterion(out.view(-1, out.size(-1)), tgt_y.view(-1))
        # 计算梯度
        loss.backward()
        # 更新参数
        optimizer.step()

        total_loss += loss.item()
        
    print("epoch: {}, loss: {}".format(epoch, total_loss / len(x_train_tensor)))
    # 计算训练集上的准确率
    model.eval()
    with torch.no_grad():
        src = x_train_tensor
        tgt = y_train_tensor
        tgt_y = decoder_output_train_tensor

        out = model(src, tgt)
        out = model.predictor(out)
        out = out.argmax(dim=-1)
        acc = (out == tgt_y).float().mean()
        print("train acc: {}".format(acc))
    # 计算测试集上的准确率
    with torch.no_grad():
        src = x_test_tensor
        tgt = y_test_tensor
        tgt_y = decoder_output_test_tensor

        out = model(src, tgt)
        out = model.predictor(out)
        out = out.argmax(dim=-1)
        acc = (out == tgt_y).float().mean()
        print("test acc: {}".format(acc))

In [ ]:
test_string = '白日依山盡，黃河入海流。'


# 把 test_string 每个字之间添加空格
test_string = ' '.join(test_string)
# 前后加上开始和结束标志
# test_string = '<SOS> ' + test_string
# 把 test_string 转化为token
test_string_token = tokenizer.texts_to_sequences([test_string])
# 截取后10个字
test_string_token = test_string_token[0][-10:]
# 转化为 numpy，补齐
test_string_mat = pad_sequences([test_string_token],maxlen=10,padding='post',truncating='post')
# test_string_mat 转化为 tensor
test_string_tensor = torch.tensor(test_string_mat).to(device)
test_string_tensor

In [ ]:
decoded = decode(model, 
       src=test_string_tensor,  
       max_iter=100, 
       SOS_token=SOS_token, 
       EOS_token=EOS_token, 
       PAD_token=PAD_token)
decoded

In [ ]:
# 把 tgt 中的数字转化为文字
tokenizer.sequences_to_texts(decoded.cpu().numpy())